In [1]:
gps_coords = 46.500458, 8.052669
year = 2020
month = 1
day = 1


In [2]:
from utils import *
import plotly.graph_objects as go
import plotly.io as pio

date = {'year':year,'month':month,'day':day}

gps_lat, gps_lon = gps_coords

interval = 30
n_intervals = 5

# start_date = datetime.datetime.now().date()

start_date = datetime.date(year = date['year'], month = date['month'], day = date['day'])

td = datetime.timedelta(days = interval)
final_date = start_date + n_intervals * td

metadata_links = {'2m' : '/Users/george-birchenough/Downloads/ch.swisstopo.swissalti3d-fGQ3d2A6.csv' , \
                '0.5m' : 'https://ogd.swisstopo.admin.ch/resources/ch.swisstopo.swissalti3d-3TuKAiHo.csv' }

transformer = Transformer.from_crs( 'epsg:4326', 'epsg:2056' )
swiss_topo_lon, swiss_topo_lat = transformer.transform( gps_lat, gps_lon)

filename = metadata_links['2m']
grid_size = 2
radius = 1
tile_meta_df = get_tile_metadata(filename)
target_tiles = get_targets(swiss_topo_lat, swiss_topo_lon, tile_meta_df, radius)
# plot_tile_corners(target_tiles)

array, blank_array = get_tiles(target_tiles)

observer_pixel, observer_height = get_observer_position(array, blank_array, swiss_topo_lat, swiss_topo_lon )
peaks_df = get_peaks( array, observer_pixel, observer_height, grid_size)


# fig = plot_main(gps_lat, gps_lon, observer_height, peaks_df, start_date, final_date, td )

Opening tile number  0  of  4
Opening tile number  1  of  4
Opening tile number  2  of  4
Opening tile number  3  of  4
Done.
Merging ... 
Done.


In [3]:
mdf_list, tdf_list, amdf_list = [], [], []
for month in np.arange(1,13):
    year = 2021
    day = 1
    date = {'year':year,'month':month,'day':day}
    start_date = datetime.date(year = date['year'], month = date['month'], day = date['day'])
    mdf_,tdf_,amdf_  = get_data(gps_coords, observer_height, peaks_df, start_date)
    mdf_list.append(mdf_)
    tdf_list.append(tdf_)
    amdf_list.append(amdf_)


In [4]:

def make_line_dict(x, y, color = 'black', width = .5):
    return {'line': {'color': color,
            'dash': 'solid',
            'width': width},
            'mode': 'lines',
            'name': '_line0',
            'showlegend' : False,
            'x': x,
            'xaxis': 'x',
            'y': y,
            'yaxis': 'y',
            'type': 'scatter'}

def make_annotation_dict(x, y, text, xshift):
    return  {'text':text, 'x':float(x), 'y':float(y), 
            'xanchor':'center', 
            'yanchor':'middle', 
            # 'height':30,
            'xshift':10*xshift,
            'yshift':-20,
            'showarrow':False
            } 

def get_sun_lines(mdf):
    lines = []
    x = mdf.loc[ mdf.daylight == 'day' ].bearing
    y = mdf.loc[ mdf.daylight == 'day' ].elevation
    lines.append(make_line_dict(x, y, color = 'gold', width = .5))

    # x = mdf.loc[ mdf.daylight == 'day' ].bearing
    # y = mdf.loc[ mdf.daylight == 'day' ].elevation
    # f = sns.lineplot(x = x, y = y , color = 'gold', linewidth=4)

    x = mdf.loc[ mdf.sunlight == 'morning_twighlight' ].azimuth
    y = mdf.loc[ mdf.sunlight == 'morning_twighlight' ].elevation
    # sns.lineplot(x = x, y = y , color = 'royalblue', linewidth=1)
    lines.append(make_line_dict(x, y, color = 'royalblue', width = .5))


    x = mdf.loc[ mdf.sunlight == 'evening_twighlight' ].azimuth
    y = mdf.loc[ mdf.sunlight == 'evening_twighlight' ].elevation
    # sns.lineplot(x = x, y = y , color = 'royalblue', linewidth=1)
    lines.append(make_line_dict(x, y, color = 'royalblue', width = .5))

    for epoch in mdf.epoch.unique():
        if epoch%2 == 1:
            x = mdf.loc[ mdf.epoch == epoch ].azimuth
            y = mdf.loc[ mdf.epoch == epoch ].elevation
            # sns.lineplot(x = x, y = y , color = 'gold', linewidth=4)
            lines.append(make_line_dict(x, y, color = 'gold', width = 2))

    return lines            

def make_solmap(month):
    mdf = mdf_list[month]
    tdf = tdf_list[month]
    peak_lines = make_line_dict(mdf.bearing, mdf.peak_angle, color='black', width = 0.5)

    pio.templates.default = "simple_white"

    fig = go.Figure(
        data=peak_lines,
    )

    annotations = []
    gdf = tdf.groupby('time')
    for hour, df in gdf:
        df = df.sort_values('date').reset_index()
        # print(df.head)
        x = float( df.azimuth )
        y = float( df.elevation )
        text = hour
        grad = float( df.grad )
        fig.add_trace(
            go.Scatter(
                x=[ x, x + 2 * grad],
                y=[ y, y - 5] ,
                line = dict( color="black", width=.1), 
                marker = None,
                mode = 'lines',
                showlegend = False
            )
        )
        # markers.append(make_marker_dict(x,y,hour))
        annotations.append( make_annotation_dict(x, y, text, grad) )

    # fig.update_annotations(annotations)
    fig.add_traces(
        get_sun_lines(mdf)
    )

    fig.update_layout( 
        annotations = annotations,
        xaxis = dict(
            tickmode = 'array',
            tickvals = [45.0, 67.5, 90.0, 112.5, 135.0, 157.5, 180.0, 202.5, 225.0, 247.5, 270.0] ,
            ticktext = ['NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W'] ),
        yaxis = dict(
            range = [-10, 80]
        ) )
    return fig



In [5]:
from pkg_resources import get_platform
from utils import *
from plot_utils import *

import plotly as py
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    html.Div( [dcc.Dropdown(id='date-select', options=[ {'label': i, 'value': i} for i in np.arange(1,12)],
                             style={'width': '140px'})] ), \
    dcc.Graph(id='plot1' )
    ])

@app.callback(
    Output('plot1', 'figure'),
    Input('date-select', 'value'))    
def update_graph(month):
    fig = make_solmap(month)
    return fig

# if __name__ == '__main__':
#     app.run_server(debug=False)
app.server.run(debug=True)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/george-birchenough/opt/anaconda3/envs/solmap/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/george-birchenough/opt/anaconda3/envs/solmap/lib/python3.7/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/Users/george-birchenough/opt/anaconda3/envs/solmap/lib/python3.7/site-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/Users/george-birchenough/opt/anaconda3/envs/solmap/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/Users/george-birchenough/opt/anaconda3/envs/solmap/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, sel

SystemExit: 1

/Users/george-birchenough/opt/anaconda3/envs/solmap/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
